In [ ]:
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-843533b3-31d4-49f3-aec1-18fb437ef9c2',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'hZ_NjVVnMgNQvLCH-U9qE3THdy8oG-bgP0ELmbYThz6Z'
}

configuration_name = 'os_3362a26b7d834f86be1984c8cd9e69e9_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json

In [9]:
in_schema = "artist STRING, auth STRING, firstName STRING, gender STRING, itemInSession INT, lastName STRING," \
            "length DOUBLE, level STRING, location STRING, method STRING, page STRING, registration LONG," \
            "sessionId INT, song STRING, status int, ts LONG, userAgent STRING, userId STRING"

df_raw = spark.read.json(cos.url('medium-sparkify-event-data.json', 'sparkify-donotdelete-pr-wxvl4d9mrqe9ee'), schema=in_schema)\
    .withColumn('iuid', F.col('userId').cast('long')).drop('userId').withColumnRenamed('iuid', 'userId')

df_raw.take(5)

[Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId=293),
 Row(artist="John Brown's Body", auth='Logged In', firstName='Sawyer', gender='M', itemInSession=74, lastName='Larson', length=380.21179, level='free', location='Houston-The Woodlands-Sugar Land, TX', method='PUT', page='NextSong', registration=1538069638000, sessionId=97, song='Bulls', status=200, ts=1538352025000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId=98),
 Row(artist='Afroman', auth='Logged In', firstName='Maverick', gender='M', item

In [11]:
df_explore = df_raw
df_explore.head()

Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId=293)

In [27]:
import sys
!{sys.executable} -m pip install plotly

    100% |████████████████████████████████| 12.9MB 987kB/s eta 0:00:01
  Stored in directory: /home/spark/shared/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
pytest-doctestplus 0.7.0 has requirement pytest>=4.0, but you'll have pytest 3.10.1 which is incompatible.
pytest-astropy 0.8.0 has requirement pytest>=4.6, but you'll have pytest 3.10.1 which is incompatible.


In [36]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df_cleaned = df_raw.dropna(how='any', subset=['userId', ])

# model.py
instead of uploading model.py it is easier here to just paste the contents

In [5]:
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Transformer


class UserLogTransformer(Transformer):
    """
    Custom transformer that aggregates Spotify log data into user features for predicting
    if they are at risk of churning.

    Extracted user information are:

    * user base data:
        * gender
        * level
        * registration (time tamp)
    * aggregated log data
        * avg_session_events  mean number of log events per session
        * min_session_events
        * max_session_events
        * total_session_events
        * ts_min  timestamp of earliest log event
        * ts_max  timestamp of most recent log event
        * period  interval in days between earliest and latest log event
        * counts of log event types (pages)
            * About
            * Add Friend
            * Add to Playlist
            * Error
            * Help
            * Home
            * Logout
            * NextSong
            * Roll Advert
            * Save Settings
            * Settings
            * Thumbs Down
            * Thumbs Up
            * Upgrade
            * Submit Downgrade
            * Submit Upgrade
        * frequencies of log event types defined as count / period for the events above
        * 1-hot encoded list of operating systems seen on userId
        * 1-hot encoded list of browser engines seen on userId

    Events interpreted as churn are deleted from the dataset:
    * Cancellation Confirmation

    Churn-related events that are deleted to avoid data leakage:
    * Cancel
    """

    def _transform(self, dataset):
        # splitting up operations into a few temporary data tables for readability
        # session statistics
        df_session_stats = dataset.select('userId', 'sessionId', 'itemInSession').groupBy('userId', 'sessionId') \
            .count().fillna(0, 'count').withColumnRenamed("count", "_count") \
            .groupBy('userId').agg(F.mean("_count").cast('double').alias("avg_session_events"),
                                   F.min("_count").cast('double').alias("min_session_events"),
                                   F.max("_count").cast('double').alias("max_session_events"),
                                   F.sum('_count').cast('double').alias("total_session_events"))
        # user interaction period stats and per user stats
        df_user_categories = dataset.select('userId', F.when(dataset.gender == 'f', 1.0).otherwise(0.0).alias('_gender'),
                                            F.when(dataset.level == 'paid', 1.0).otherwise(0.0).alias('_level'))\
            .groupBy('userId').agg(F.first('_gender').alias('gender'), F.max('_level').alias('maxLevel'),
                                   (F.max('_level') - F.min('_level')).alias('changedLevel'))
        df_by_user = dataset.select('userId', 'ts', 'registration').groupBy('userId')\
            .agg(#F.min('ts').cast('double').alias("ts_min"),
                 #F.max('ts').cast('double').alias("ts_max"),
                 F.first('registration').cast('double').alias('registration'),
                 ((F.max('ts')-F.min('ts')) / (3600 * 24 * 1000)).alias('period')
                 )
        # user browser and operating system counts
        df_os_browser = dataset.select('userId', 'userAgent',
                                       F.regexp_replace(F.regexp_extract(F.col('userAgent'), ".*?(\(.*\)).*", 1),
                                        '[\(\);:;\s\/.,]+', '').alias('os'),
                                       F.regexp_replace(F.regexp_extract(F.col('userAgent'), ".*\s(.*)", 1),
                                        '[\(\);:;\s\/.,]+', '').alias('browser'))
        df_os_onehot = df_os_browser.groupBy('userId').pivot('os')\
            .agg(F.countDistinct('userId').cast('double').alias('os')).fillna(0)
        df_browser_onehot = df_os_browser.groupBy('userId').pivot('browser') \
            .agg(F.countDistinct('userId').cast('double').alias('browser')).fillna(0)
        # user page counts and frequencies
        df_page_counts = dataset.select('userId', F.column('page').alias('page'))\
            .join(df_by_user.select('userId', 'period'), on='userId')\
            .groupBy('userId').pivot('page').agg(F.count('userId').cast('double').alias('count'),
                                                 (F.count('userId') / F.first('period')).cast('double')
                                                 .alias('freq')).fillna(0)\
            .drop('Cancel_count', 'Cancellation Confirmation_count', 'Submit Downgrade_count',
                  'Cancel_freq', 'Cancellation Confirmation_freq', 'Submit Downgrade_freq')
        return df_by_user.join(df_page_counts, on='userId')\
            .join(df_user_categories, on='userId')\
            .join(df_os_onehot, on='userId')\
            .join(df_browser_onehot, on='userId')\
            .join(df_session_stats, on='userId')


class LogCleanTransformer(Transformer):
    """
    Custom Transformer that cleans user log data for machine learning.

    Drops all rows with nan userId values
    """

    def _transform(self, dataset):
        return dataset.dropna(how='any', subset=['userId', ])


class UserLabelTransformer(Transformer):
    """
    Custom Transformer that returns if a user has churn events in his log history

    Defined Churn events are:
    * Cancellation Confirmation
    * Submit Downgrade
    """

    def _transform(self, dataset):
        return dataset.select('userId', F.when((dataset.page == 'Cancellation Confirmation'),
                                               1).otherwise(0).alias('churn'))\
            .groupBy('userId').agg(F.max('churn').alias('churned'))


class TrainingAssembler(Transformer):
    """"
    Assemble the data set for training into label and feature vector columns.

    Expects a dataframe of numeric columns, one of which should be named 'churned'. The 'churned' column is used as the
    labels columns. All other columns are assembled into the features vector.
    """

    def _transform(self, dataset):
        input_cols = dataset.columns
        input_cols.remove('churned')
        vectassemble = VectorAssembler(inputCols=input_cols, outputCol='features', handleInvalid='skip')
        return vectassemble.transform(dataset).select(
            F.column('churned').alias('label'), 'features', 'userId'
        )


class MasterTransformer(Transformer):
    """
    Transformer instance that puts together the building blocks for label and feature extraction
    """

    def _transform(self, dataset):
        logtransform = UserLogTransformer()
        labeltransform = UserLabelTransformer()
        assembler = TrainingAssembler()
        return assembler.transform(logtransform.transform(dataset).join(labeltransform.transform(dataset), on='userId'))


# Modeling
Here we use the insights gained fromt he exploration ont he small sample of the full data set. Therefore, here the data is only split up inti a training and test set. Validation will be skipped, since we will beusing the classification method and hyperparameters obtained there. I.e. we use a linear support vector machine with maximum iterations set to 300 and and a regularization parameter of 1E-4.

## Preparation
Import required classes, transform log data to features and split off test data. Will use 10% of userIds as test dataset. The remainder will be used for training.


In [6]:
from pyspark.ml.classification import LinearSVC

In [13]:
m_tra = MasterTransformer()
cleaner = LogCleanTransformer()
mt = m_tra.transform(cleaner.transform(df_raw))

In [14]:
mt.head(30)
train, test = mt.randomSplit([0.90, 0.10], seed=1404)

In [15]:
lsvc = LinearSVC(maxIter=300, regParam=1e-4)
model_lsvc = lsvc.fit(train)


In [17]:
pred_lsvc = model_lsvc.transform(test).withColumn('tp', F.col('label') * F.col('prediction'))\
    .withColumn('fp', (1 - F.col('label')) * F.col('prediction'))\
    .withColumn('fn', F.col('label') * (1 - F.col('prediction')))

In [18]:
tp_sum = pred_lsvc.agg(F.sum('tp').alias('tp_s')).collect()[0]['tp_s']
fp_sum = pred_lsvc.agg(F.sum('fp').alias('fp_s')).collect()[0]['fp_s']
fn_sum = pred_lsvc.agg(F.sum('fn').alias('fn_s')).collect()[0]['fn_s']
F1_rf = tp_sum / (tp_sum + 0.5 * (fp_sum + fn_sum))
print("Linear SVC regression F1 score :", F1_rf)

Linear SVC regression F1 score : 0.9230769230769231


In [19]:
print(model_lsvc.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2) (default: 2)
featuresCol: features column name (default: features)
fitIntercept: whether to fit an intercept term (default: True)
labelCol: label column name (default: label)
maxIter: maximum number of iterations (>= 0) (default: 100, current: 300)
predictionCol: prediction column name (default: prediction)
rawPredictionCol: raw prediction (a.k.a. confidence) column name (default: rawPrediction)
regParam: regularization parameter (>= 0) (default: 0.0, current: 0.0001)
standardization: whether to standardize the training features before fitting the model (default: True)
threshold: threshold in binary classification prediction applied to rawPrediction (default: 0.0)
tol: the convergence tolerance for iterative algorithms (>= 0) (default: 1e-06)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0 (undefined)


In [37]:
model_lsvc.write().overwrite().save(cos.url('lsvc_model', 'sparkify-donotdelete-pr-wxvl4d9mrqe9ee'))

In [39]:
pred_lsvc.coalesce(1).write.mode("overwrite").json(cos.url('lsvc-prediction.json', 'sparkify-donotdelete-pr-wxvl4d9mrqe9ee'))

In [40]:
test.coalesce(1).write.json(cos.url('testdata.json', 'sparkify-donotdelete-pr-wxvl4d9mrqe9ee'))

In [41]:
train.coalesce(1).write.json(cos.url('traindata.json', 'sparkify-donotdelete-pr-wxvl4d9mrqe9ee'))